In [2]:
import os
import zipfile
import random
import shutil
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
import matplotlib.pyplot as plt

In [3]:
source_path = '/content/drive/MyDrive/PetImages'

source_path_dogs = os.path.join(source_path, 'Dog')
source_path_cats = os.path.join(source_path, 'Cat')

# Deletes all non-image files (there are two .db files bundled into the dataset)
!find /content/drive/MyDrive/PetImages -type f ! -name "*.jpg" -exec rm {} +

# os.listdir returns a list containing all files under the given path
print(f"There are {len(os.listdir(source_path_dogs))} images of dogs.")
print(f"There are {len(os.listdir(source_path_cats))} images of cats.")

There are 12500 images of dogs.
There are 12500 images of cats.


In [4]:
root_dir = '/tmp/cats-v-dogs'

# Empty directory to prevent FileExistsError is the function is run several times
if os.path.exists(root_dir):
  shutil.rmtree(root_dir)

# GRADED FUNCTION: create_train_val_dirs
def create_train_val_dirs(root_path):
  """
  Creates directories for the train and test sets

  Args:
    root_path (string) - the base directory path to create subdirectories from

  Returns:
    None
  """
  ### START CODE HERE

  # HINT:
  # Use os.makedirs to create your directories with intermediate subdirectories
  # Don't hardcode the paths. Use os.path.join to append the new directories to the root_path parameter
  subdirectories = ["training", "training/cats", "training/dogs", "validation", "validation/cats", "validation/dogs"]

  for subdirectory in subdirectories:
      full_path=os.path.join(root_dir, subdirectory)
      os.makedirs(full_path)

  pass

  ### END CODE HERE
try:
  create_train_val_dirs(root_path=root_dir)
except FileExistsError:
  print("You should not be seeing this since the upper directory is removed beforehand")

In [5]:
for rootdir, dirs, files in os.walk(root_dir):
    for subdir in dirs:
        print(os.path.join(rootdir, subdir))

/tmp/cats-v-dogs/training
/tmp/cats-v-dogs/validation
/tmp/cats-v-dogs/training/cats
/tmp/cats-v-dogs/training/dogs
/tmp/cats-v-dogs/validation/cats
/tmp/cats-v-dogs/validation/dogs


In [19]:
#def split_data(SOURCE_DIR, TRAINING_DIR, VALIDATION_DIR, SPLIT_SIZE):

CAT_SOURCE_DIR = "/content/drive/MyDrive/PetImages/Cat"
DOG_SOURCE_DIR = "/content/drive/MyDrive/PetImages/Dog"

dog_files = [file for file in os.listdir(DOG_SOURCE_DIR) if os.path.getsize(os.path.join(DOG_SOURCE_DIR, file)) > 0]
cat_files = [file for file in os.listdir(CAT_SOURCE_DIR) if os.path.getsize(os.path.join(CAT_SOURCE_DIR, file)) > 0]

SPLIT_SIZE = .9
cut = len(dog_files) * SPLIT_SIZE
print(len(dog_files))
print(SPLIT_SIZE)
print(cut)

12499
0.9
11249.1


In [13]:
def split_data(SOURCE_DIR, TRAINING_DIR, VALIDATION_DIR, SPLIT_SIZE):

  CAT_SOURCE_DIR = "/content/drive/MyDrive/PetImages/Cat"
  DOG_SOURCE_DIR = "/content/drive/MyDrive/PetImages/Dog"

  TRAINING_DIR = "/tmp/cats-v-dogs/training/"
  VALIDATION_DIR = "/tmp/cats-v-dogs/validation/"

  TRAINING_CATS_DIR = os.path.join(TRAINING_DIR, "cats/")
  VALIDATION_CATS_DIR = os.path.join(VALIDATION_DIR, "cats/")

  TRAINING_DOGS_DIR = os.path.join(TRAINING_DIR, "dogs/")
  VALIDATION_DOGS_DIR = os.path.join(VALIDATION_DIR, "dogs/")

  dog_files = [file for file in os.listdir(DOG_SOURCE_DIR) if os.path.getsize(os.path.join(DOG_SOURCE_DIR, file)) > 0]
  cat_files = [file for file in os.listdir(CAT_SOURCE_DIR) if os.path.getsize(os.path.join(CAT_SOURCE_DIR, file)) > 0]

  def check_zeros_dog(file_list):
    zero_sized_files_dog = [file for file in os.listdir(DOG_SOURCE_DIR) if os.path.getsize(os.path.join(DOG_SOURCE_DIR, file)) ==0]
    for file in zero_sized_files_dog:
      print(f"File with zero size: {file}")

  def check_zeros_cat(file_list):
    zero_sized_files_cat = [file for file in os.listdir(CAT_SOURCE_DIR) if os.path.getsize(os.path.join(CAT_SOURCE_DIR, file)) ==0]
    for file in zero_sized_files_cat:
      print(f"File with zero size: {file}")

  check_zeros_dog(dog_files)
  check_zeros_cat(cat_files)

  random.shuffle(dog_files)
  random.shuffle(cat_files)

  dog_split = int(len(dog_files)) * SPLIT_SIZE
  cat_split = int(len(cat_files)) * SPLIT_SIZE
  print(float(dog_split))
  print(float(cat_split))
  print(len(dog_files))
  print(len(cat_files))
  for file in dog_files[:int(dog_split)]:
    shutil.copy(os.path.join(DOG_SOURCE_DIR, file), TRAINING_DOGS_DIR)
  for file in cat_files[:int(cat_split)]:
    shutil.copy(os.path.join(CAT_SOURCE_DIR, file), TRAINING_CATS_DIR)

  for file in dog_files[dog_split:]:
    shutil.copy(os.path.join(dog_files, file), os.path.join(VALIDATION_DOGS_DIR))
  for file in cat_files[cat_split:]:
    shutil.copy(os.path.join(cat_files, file), os.path.join(VALIDATION_CATS_DIR))

  print("Data split bitches!")


In [14]:
CAT_SOURCE_DIR = "/content/drive/MyDrive/PetImages/Cat/"
DOG_SOURCE_DIR = "/content/drive/MyDrive/PetImages/Dog/"

TRAINING_DIR = "/tmp/cats-v-dogs/training/"
VALIDATION_DIR = "/tmp/cats-v-dogs/validation/"

TRAINING_CATS_DIR = os.path.join(TRAINING_DIR, "cats/")
VALIDATION_CATS_DIR = os.path.join(VALIDATION_DIR, "cats/")

TRAINING_DOGS_DIR = os.path.join(TRAINING_DIR, "dogs/")
VALIDATION_DOGS_DIR = os.path.join(VALIDATION_DIR, "dogs/")

# Empty directories in case you run this cell multiple times
if len(os.listdir(TRAINING_CATS_DIR)) > 0:
  for file in os.scandir(TRAINING_CATS_DIR):
    os.remove(file.path)
if len(os.listdir(TRAINING_DOGS_DIR)) > 0:
  for file in os.scandir(TRAINING_DOGS_DIR):
    os.remove(file.path)
if len(os.listdir(VALIDATION_CATS_DIR)) > 0:
  for file in os.scandir(VALIDATION_CATS_DIR):
    os.remove(file.path)
if len(os.listdir(VALIDATION_DOGS_DIR)) > 0:
  for file in os.scandir(VALIDATION_DOGS_DIR):
    os.remove(file.path)

# Define proportion of images used for training
split_size = .9

# Run the function
# NOTE: Messages about zero length images should be printed out
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, VALIDATION_DOGS_DIR, split_size)
split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, VALIDATION_CATS_DIR, split_size)


# Check that the number of images matches the expected output

# Your function should perform copies rather than moving images so original directories should contain unchanged images
print(f"\n\nOriginal cat's directory has {len(os.listdir(CAT_SOURCE_DIR))} images")
print(f"Original dog's directory has {len(os.listdir(DOG_SOURCE_DIR))} images\n")

# Training and validation splits
print(f"There are {len(os.listdir(TRAINING_CATS_DIR))} images of cats for training")
print(f"There are {len(os.listdir(TRAINING_DOGS_DIR))} images of dogs for training")
print(f"There are {len(os.listdir(VALIDATION_CATS_DIR))} images of cats for validation")
print(f"There are {len(os.listdir(VALIDATION_DOGS_DIR))} images of dogs for validation")

KeyboardInterrupt: ignored

In [ ]:
def split_data(SOURCE_DIR, TRAINING_DIR, VALIDATION_DIR, SPLIT_SIZE):

  CAT_SOURCE_DIR = "/content/drive/MyDrive/PetImages"
  DOG_SOURCE_DIR = "/content/drive/MyDrive/PetImages"

  TRAINING_DIR = "/tmp/cats-v-dogs/training/"
  VALIDATION_DIR = "/tmp/cats-v-dogs/validation/"

  TRAINING_CATS_DIR = os.path.join(TRAINING_DIR, "cats/")
  VALIDATION_CATS_DIR = os.path.join(VALIDATION_DIR, "cats/")

  TRAINING_DOGS_DIR = os.path.join(TRAINING_DIR, "dogs/")
  VALIDATION_DOGS_DIR = os.path.join(VALIDATION_DIR, "dogs/")



  dog_files = [file for file in os.listdir(DOG_SOURCE_DIR) if os.path.getsize(os.path.join(DOG_SOURCE_DIR, file)) > 0]
  cat_files = [file for file in os.listdir(CAT_SOURCE_DIR) if os.path.getsize(os.path.join(CAT_SOURCE_DIR, file)) > 0]

  random.shuffle(dog_files)
  random.shuffle(cat_files)

  dog_split = len(dog_files) * SPLIT_SIZE
  cat_split = len(cat_files) * SPLIT_SIZE

  for file in dog_files[:dog_split]:
    shutil.copy(os.path.join(dog_files, file), os.path.join(TRAINING_DIR, 'Dog'))
  for file in cat_files[:cat_split]:
    shutil.copy(os.path.join(cat_files, file), os.path.join(TRAINING_DIR, 'Cat'))

  for file in dog_files[dog_split:]:
    shutil.copy(os.path.join(dog_files, file), os.path.join(VALIDATION_DIR, 'Dog'))
  for file in cat_files[cat_split:]:
    shutil.copy(os.path.join(cat_files, file), os.path.join(VALIDATION_DIR, 'Cat'))

  print("Data split bitches!")

  for filename in random.sample(DOG_SOURCE_DIR, len(DOG_SOURCE_DIR)):
    if len(os.listdir(TRAINING_DOGS_DIR)) > dog_split_count:
      if os.path.getsize(os.path.join(DOG_SOURCE_DIR, filename)) == 0:
        print("{filename} is too small"),
      else:
        copyfile(os.path.join(DOG_SOURCE_DIR, filename), os.path.join(TRAINING_DOGS_DIR, filename))
    if os.path.getsize(os.path.join(DOG_SOURCE_DIR, filename)) == 0:
      print("{filename} is too small"),
    else:
      copyfile(os.path.join(DOG_SOURCE_DIR, filename), os.path.join(VALIDATION_DOGS_DIR, filename))

  cat_split_count = len(os.listdir(CAT_SOURCE_DIR)) * SPLIT_SIZE

  for filename in random.sample(CAT_SOURCE_DIR, len(CAT_SOURCE_DIR)):
    if len(os.listdir(TRAINING_CATS_DIR)) > cat_split_count:
      if os.path.getsize(os.path.join(CAT_SOURCE_DIR, filename)) == 0:
        print("{filename} is too small"),
      else:
        copyfile(os.path.join(CAT_SOURCE_DIR, filename), os.path.join(TRAINING_CATS_DIR, filename))
    if os.path.getsize(os.path.join(CAT_SOURCE_DIR, filename)) == 0:
      print("{filename} is too small"),
    else:
      copyfile(os.path.join(CAT_SOURCE_DIR, filename), os.path.join(VALIDATION_CATS_DIR, filename))

In [ ]:
def split_data(SOURCE_DIR, TRAINING_DIR, VALIDATION_DIR, SPLIT_SIZE):
  """
  Splits the data into train and test sets

  Args:
    SOURCE_DIR (string): directory path containing the images
    TRAINING_DIR (string): directory path to be used for training
    VALIDATION_DIR (string): directory path to be used for validation
    SPLIT_SIZE (float): proportion of the dataset to be used for training

  Returns:
    None
  """
  ### START CODE HERE

  CAT_SOURCE_DIR = "/content/drive/MyDrive/PetImages"
  DOG_SOURCE_DIR = "/content/drive/MyDrive/PetImages"

  zero_size_images_dog = []

  def find_zero_size_images_dog(directory):
    #zero_size_images_dog = []
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        if os.path.getsize(filepath) == 0:
            zero_size_images_dog.append(filename)
            #copyfile(os.path.join(DOG_SOURCE_DIR, filename), os.path.join(zero_dogs, filename))
  #  return zero_size_images_dog

  directory_path = DOG_SOURCE_DIR
  zero_size_images_list_dog = find_zero_size_images_dog(directory_path)

  zero_size_images_cat = []

  def find_zero_size_images_cat(directory):
    zero_size_images_cat = []
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        if os.path.getsize(filepath) == 0:
            zero_size_images_cat.append(filename)
            #copyfile(os.path.join(CAT_SOURCE_DIR, filename), os.path.join(zero_cats, filename))
  #  return zero_size_images_cat

  directory_path = CAT_SOURCE_DIR
  zero_size_images_list_cat = find_zero_size_images_cat(directory_path)

#Dogs

  dog_split_count = (len(os.listdir(DOG_SOURCE_DIR)) - len(zero_size_images_dog)) * SPLIT_SIZE
for filename in random.sample(DOG_SOURCE_DIR, len(DOG_SOURCE_DIR)):
    if not len(os.listdir(DOG_SOURCE_DIR)) <= dog_split_count:
     if not os.path.getsize(os.path.join(DOG_SOURCE_DIR, filename)) == 0:
        if not os.path.isfile(os.path.join(DOG_SOURCE_DIR, filename)):
          if not os.path.isfile(os.path.join(VALIDATION_DIR, filename)):
            copyfile(os.path.join(DOG_SOURCE_DIR, filename), os.path.join(VALIDATION_DIR, filename))
  for filename in random.sample(DOG_SOURCE_DIR, len(DOG_SOURCE_DIR)):
    if not len(os.listdir(DOG_SOURCE_DIR)) > dog_split_count:
      if os.path.getsize(os.path.join(DOG_SOURCE_DIR, filename)) == 0:
        print("{filename} is too small"),
        if not os.path.isfile(os.path.join(TRAINING_DIR, filename)):
          copyfile(os.path.join(DOG_SOURCE_DIR, filename), os.path.join(TRAINING_DIR, filename))
  for filename in random.sample(DOG_SOURCE_DIR, len(DOG_SOURCE_DIR)):
    if not len(os.listdir(DOG_SOURCE_DIR)) <= dog_split_count:
     if not os.path.getsize(os.path.join(DOG_SOURCE_DIR, filename)) == 0:
        if not os.path.isfile(os.path.join(DOG_SOURCE_DIR, filename)):
          if not os.path.isfile(os.path.join(VALIDATION_DIR, filename)):
            copyfile(os.path.join(DOG_SOURCE_DIR, filename), os.path.join(VALIDATION_DIR, filename))
  for filename in zero_size_images_dog:
    print(filename)

#CAts
  cat_split_count = (len(os.listdir(CAT_SOURCE_DIR)) - len(os.listdir(zero_cats))) * SPLIT_SIZE
  for filename in random.sample(CAT_SOURCE_DIR, len(CAT_SOURCE_DIR)):
    if not len(os.listdir(CAT_SOURCE_DIR)) > cat_split_count:
      if not os.path.getsize(os.path.join(DOG_SOURCE_DIR, filename)) == 0:
        if not os.path.isfile(os.path.join(TRAINING_DIR, filename)):
          copyfile(os.path.join(CAT_SOURCE_DIR, filename), os.path.join(TRAINING_DIR, filename))
  for filename in random.sample(CAT_SOURCE_DIR, len(CAT_SOURCE_DIR)):
    if not len(os.listdir(CAT_SOURCE_DIR)) <= cat_split_count:
      if not os.path.getsize(os.path.join(DOG_SOURCE_DIR, filename)) == 0:
        if not os.path.isfile(os.path.join(CAT_SOURCE_DIR, filename)):
          if not os.path.isfile(os.path.join(VALIDATION_DIR, filename)):
            copyfile(os.path.join(CAT_SOURCE_DIR, filename), os.path.join(VALIDATION_DIR, filename))
  for filename in zero_size_images_cat:
    print(filename)

pass

#CAT_SOURCE_DIR = os.path.join(SOURCE_DIR, "Cat")
  #DOG_SOURCE_DIR = os.path.join(SOURCE_DIR, "Dog")

  #os.makedirs(TRAINING_DIR, exist_ok=True)
  #os.makedirs(VALIDATION_DIR, exist_ok=True)

  #zero_dogs = "/tmp/ZeroDogs/"
  #os.makedirs(zero_dogs)

  #zero_cats = "/tmp/ZeroCats/"
  #os.makedirs(zero_cats)
  ### END CODE HERE

IndentationError: ignored

In [ ]:
dog_split = int(len(DOG_SOURCE_DIR)) * SPLIT_SIZE
  cat_split = int(len(CAT_SOURCE_DIR)) * SPLIT_SIZE

  dogs = os.listdir(source_path_dogs)

  for file in random.shuffle(os.listdir(source_path_dogs)):
    if os.path.getsize(os.path.join(DOG_SOURCE_DIR, file)) == 0:
      print("{file} is too teeny")
    if len(os.listdir(os.path.join(TRAINING_DOGS_DIR))) > dog_split:
      shutil.copy(os.path.join(DOG_SOURCE_DIR, file), TRAINING_DOGS_DIR)
    else:
        shutil.copy(os.path.join(DOG_SOURCE_DIR, file), VALIDATION_DOGS_DIR)

  cats = os.listdir(source_path_cats)

  for file in random.shuffle(os.listdir(source_path_cats)):
    if os.path.getsize(os.path.join(CAT_SOURCE_DIR, file)) == 0:
      print("{file} is too teeny")
    if len(os.listdir(os.path.join(TRAINING_CATS_DIR))) > cat_split:
      shutil.copy(os.path.join(CAT_SOURCE_DIR, file), TRAINING_CATS_DIR)
    else:
      shutil.copy(os.path.join(CAT_SOURCE_DIR, file), VALIDATION_CATS_DIR)